## Chapter 17.2 전이학습

In [1]:
versioninfo()

Julia Version 1.9.3
Commit bed2cd540a1 (2023-08-24 14:43 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 20 × 12th Gen Intel(R) Core(TM) i7-12700F
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, alderlake)
  Threads: 2 on 20 virtual cores
Environment:
  LD_LIBRARY_PATH = :/usr/local/cuda-11.7/lib64


In [2]:
pwd()

"/home/bread"

In [4]:
using Pkg
Pkg.activate("~/test")
Pkg.instantiate()

  Activating project at `~/~/test`
Precompiling project...
  ✓ StringEncodings
  ✓ ImageMagick_jll
  ✓ HDF5_jll
  ✓ LightXML
  ✓ Pickle
  ? WordTokenizers
  ? Fetch
  ? ImageCore
  ✓ Embeddings
  ✓ HDF5
  ✓ MAT
  ✓ Snowball
  ? PrimitiveOneHot
  ✓ TextAnalysis
  ? Metalhead
  ✓ TextEncodeBase
  ✓ BytePairEncoding
  ? NeuralAttentionlib
  ? ImageDraw
  ? PNGFiles
  ? ImageMagick
  ? ImageMorphology
  ? ImageBase
  ✓ Sixel
  ? ImageTransformations
  ✓ Zygote → ZygoteDistancesExt
  ✓ JpegTurbo
  ? ImageFiltering
  ? ImageAxes
  ✓ ImageContrastAdjustment
  ✓ ImageMetadata
  ✓ ImageShow
  ✓ Netpbm
  ✓ ImageIO
  ✓ MLDatasets
  ✓ ImageDistances
  ? ImageSegmentation
  ? Transformers
  ✓ ImageQualityIndexes
  ✓ Images
  ✓ ObjectDetector
  25 dependencies successfully precompiled in 109 seconds. 331 already precompiled.
  16 dependencies failed but may be precompilable after restarting julia
  18 dependencies had warnings during precompilation:
┌ Metalhead [dbeba491-748d-5e0e-a39e-b530a07fa0cc]

In [5]:
import Metalhead, Images
using StatsBase: sample, shuffle
import Flux, NNlib
import Zygote, Optimisers, Functors
using Formatting: printfmtln
using Random: MersenneTwister

[ Info: Precompiling Metalhead [dbeba491-748d-5e0e-a39e-b530a07fa0cc]
[ Info: Precompiling Images [916415d5-f1e6-5110-898d-aaa5f9f070e0]
[ Info: Precompiling ZygoteColorsExt [e68c091a-8ea5-5ca7-be4f-380657d4ad79]
[ Info: Precompiling ZygoteDistancesExt [5865c103-18d1-586a-9b11-010bbc2260a8]


In [6]:
function get_image_sampler(path, rng)
    if isempty(readdir(path))
        error("Empty train folder")
    end
    files = joinpath.(path, readdir(path))    
    dogs = filter(x -> occursin("dog", x), files)
    cats = filter(x -> occursin("cat", x), files) 

    function image_sampler(n = 10, size = (224, 224))
        @assert iseven(n)
        dogs_ = sample(rng, dogs, Int(n/2))
        cats_ = sample(rng, cats, Int(n/2))
        imgs_paths = shuffle(rng, vcat(dogs_, cats_))

        imgs = Images.load.(imgs_paths)
        imgs = map(img -> Images.imresize(img, size...), imgs)
        
        imgs = map(imgs) do img # [CHW] -> [WHC]
            permutedims(Images.channelview(img), (3,2,1))
        end
        imgs = cat(imgs..., dims = 4) # [WHC] => WHCN
        imgs = Float32.(imgs)

        labels = map(x -> occursin("dog", x) ? 1 : 0, imgs_paths)
        labels = Flux.onehotbatch(labels, [0,1])

        imgs, labels
    end
end
     

get_image_sampler (generic function with 1 method)

In [7]:
function train(loader, model, loss_fn, optimizer)
    num_batches = length(loader)
    Flux.testmode!(model, false)
    for (batch, (X, y)) in enumerate(loader)
        X, y = Flux.gpu(X), Flux.gpu(y)
        grad = Zygote.gradient(m -> loss_fn(m, X, y), model)[1]
        optimizer, model = Optimisers.update(optimizer, model, grad)
        if batch % 10 == 0
            loss = loss_fn(model, X, y)
            printfmtln("[Train] loss: {:.7f} [{:>3d}/{:>3d}]", 
                loss, batch, num_batches)
        end
    end
    model, optimizer
end

function test(loader, model, loss_fn)
    num_batches = length(loader)
    Flux.testmode!(model, true)
    acc, tot = 0, 0
    loss = 0f0
    for (X, y) in loader
        X, y = Flux.gpu(X), Flux.gpu(y)
        pred = model(X)
        acc += sum(Flux.onecold(pred) .== Flux.onecold(y))
        tot += size(X)[end]
        loss += loss_fn(model, X, y)
    end
    acc, avg_loss = acc / tot * 100, loss / num_batches
    printfmtln("[Test] Accuracy: {:.1f}, Avg loss: {:.7f}", 
        acc, avg_loss)
    acc, avg_loss
end

init(rng) = Flux.glorot_uniform(rng)
     

init (generic function with 1 method)

In [8]:
struct MyResnet
    resnet
    dense
end
function (a::MyResnet)(x)
    x = a.resnet.layers[1](x)
    x = Flux.AdaptiveMeanPool((1, 1))(x)
    x = Flux.flatten(x)
    a.dense(x)
end
Functors.@functor MyResnet

In [11]:
function run_resnet(rng; pretrain)
    sampler = get_image_sampler("/home/bread/JULIA/train", rng)
    resnet = Metalhead.ResNet(18, pretrain = pretrain)
    model = MyResnet(resnet, Flux.Dense(512 => 2; init=init(rng))) 
    model = model |> Flux.gpu
    optimizer = Optimisers.setup(Optimisers.Adam(), model)
    loader = (sampler(10) for _ in 1:100) 
    loss_fn = (m, x, y) -> Flux.Losses.logitcrossentropy(m(x), y)
    model, _ = train(loader, model, loss_fn, optimizer)
    loader = (sampler(10) for _ in 1:20)
    test(loader, model, loss_fn)
end
     

run_resnet (generic function with 1 method)

In [12]:
# /home/bread/JULIA/train

rng = MersenneTwister(1)
run_resnet(rng; pretrain = false);

[ Info: Precompiling JpegTurbo [b835a17e-a41a-41e7-81f0-2f016b05efe0]


[Train] loss: 0.4889830 [ 10/100]
[Train] loss: 0.4982725 [ 20/100]
[Train] loss: 0.8082874 [ 30/100]
[Train] loss: 0.9055136 [ 40/100]
[Train] loss: 0.5314153 [ 50/100]
[Train] loss: 0.7690021 [ 60/100]
[Train] loss: 0.7094128 [ 70/100]
[Train] loss: 0.8273449 [ 80/100]
[Train] loss: 0.7165024 [ 90/100]
[Train] loss: 0.9815868 [100/100]
[Test] Accuracy: 49.5, Avg loss: 0.8633488


In [13]:
Optimisers.trainable(x::MyResnet) = (; dense = x.dense)
run_resnet(rng; pretrain = true);

[Train] loss: 0.6280646 [ 10/100]
[Train] loss: 0.3220276 [ 20/100]
[Train] loss: 0.2373100 [ 30/100]
[Train] loss: 0.1000746 [ 40/100]
[Train] loss: 0.2597194 [ 50/100]
[Train] loss: 0.0759974 [ 60/100]
[Train] loss: 0.0343409 [ 70/100]
[Train] loss: 0.2529985 [ 80/100]
[Train] loss: 0.3924147 [ 90/100]
[Train] loss: 0.2217513 [100/100]
[Test] Accuracy: 93.5, Avg loss: 0.1560094


In [14]:
rng = MersenneTwister(1)
accuracies, train_losses = run_resnet(rng; pretrain = false);

[Train] loss: 0.7451362 [ 10/100]
[Train] loss: 0.7291723 [ 20/100]
[Train] loss: 0.6813130 [ 30/100]
[Train] loss: 0.7163754 [ 40/100]
[Train] loss: 0.7101868 [ 50/100]
[Train] loss: 0.7047311 [ 60/100]
[Train] loss: 0.7832033 [ 70/100]
[Train] loss: 0.7099093 [ 80/100]
[Train] loss: 0.7633210 [ 90/100]
[Train] loss: 0.6969859 [100/100]
[Test] Accuracy: 55.0, Avg loss: 0.7148012


In [18]:
using Plots

Plots.gr(size=(400,250))
title = "unfreeze"
Plots.plot(train_losses, accuracies, title=title)

LoadError: Cannot convert Float64 to series data for plotting

In [20]:
accuracies

0.71480113f0